In [1]:
import numpy as np
import pandas as pd
from models import Hankel,Rank,Cluster,Meepc
import warnings
warnings.simplefilter('ignore')
import math
from sklearn.pipeline import Pipeline
from sklearn import set_config
from matplotlib import style
from matplotlib.patches import Ellipse
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE

In [2]:
set_config(display='diagram')

In [3]:
hankel = Hankel.Hankel()
rank = Rank.Rank()
cluster = Cluster.Cluster()
meepc = Meepc.MEEPC()

In [4]:
df = pd.read_csv('~/data/training_data.csv')
sensors = [3,9,25,43]
df = df.iloc[:, sensors]

In [5]:
df

,1_AIT_004_PV,1_MV_003_STATUS,2_FIC_401_CO,2_LS_101_AH
0,504.140,1.0,100.0000,0.0
1,504.140,1.0,100.0000,0.0
2,504.140,1.0,100.0000,0.0
3,504.140,1.0,100.0000,0.0
4,504.145,1.0,100.0000,0.0
...,...,...,...,...
971995,420.420,1.0,14.7983,0.0
971996,420.420,1.0,14.7983,0.0
971997,420.420,1.0,14.7983,0.0
971998,420.420,1.0,14.7983,0.0


In [6]:
def calc_width_height(v):
    width = 2/np.sqrt(v) 
    return width

    
def draw_ellipse(i,j,centers,widths,X):
    
    plt.xlim(centers[i]-widths[i]/2,centers[i]+widths[i]/2)
    plt.ylim(centers[j]-widths[j]/2,centers[j]+widths[j]/2)

    ellipse = Ellipse(xy=(centers[i], centers[j]), width=widths[i], height=widths[j])
    ellipse.fill = False
    ellipse.set_ec('red')
    ax.add_patch(ellipse)
    plt.scatter(x = X[::,i], y = X[::,j])
    

In [22]:
df

,1_AIT_004_PV,1_MV_003_STATUS,2_FIC_401_CO,2_LS_101_AH
0,504.140,1.0,100.0000,0.0
1,504.140,1.0,100.0000,0.0
2,504.140,1.0,100.0000,0.0
3,504.140,1.0,100.0000,0.0
4,504.145,1.0,100.0000,0.0
...,...,...,...,...
971995,420.420,1.0,14.7983,0.0
971996,420.420,1.0,14.7983,0.0
971997,420.420,1.0,14.7983,0.0
971998,420.420,1.0,14.7983,0.0


In [31]:
X = df.iloc[:,2].values
X = hankel.fit(X,30,0.5)
X=X.T
kmeans,optimal_k = cluster.fit(X)
print(optimal_k)
radiis = []
weights=[]
centers=[]
clusters_R=[]
clusters_V=[]
# tsne=TSNE(n_components=2,perplexity=30)


for i in range(optimal_k):
    cluster_ = X[np.where(kmeans.labels_ == i)[0]]
    r = rank.fit(cluster_)
    clusters_R.append(r)
    print("r: "+str(r))
    U,Sigma,VT = np.linalg.svd(cluster_)
    V = VT.T
    clusters_V.append(V)
    cluster_ = np.matmul(cluster_,V[:,:r])
    # cluster_=tsne.fit_transform(cluster_)
    weight,center = meepc.fit(cluster_)
    weights.append(weight)
    centers.append(center)
    widths=calc_width_height(weight)
    var1=np.square(cluster_-center)
    var2=np.matmul(weight,var1.T)
    max_radius=math.sqrt(np.amax(var2))
    radiis.append(max_radius)


#     #plotting
# #     ax = plt.gca()
# #     ellipse = Ellipse(xy=(center[0],center[1]), width=widths[0], height=widths[1])
# #     ellipse.fill = False 
# #     ellipse.set_ec('red')
# #     ax.add_patch(ellipse)
# #     sns.scatterplot(x = cluster_[:,0], y = cluster_[:,1])
# # plt.show()
radiis



4
r: 109
r: 104
r: 108
r: 394


[1.0061855368455646,
 0.9999991242850176,
 1.0019411049191913,
 1.0002006671389196]

In [10]:
if math.isnan(max_radius):
        print('Rank: ',r)
        print('Cluster Shape: ',cluster.shape)
        print('CLuster: ',cluster)
        print('Weight Shape: ', weight.shape)
        print('Weight: ', weight)
        print('Center Shape: ', center.shape)
        print('Center: ', center)
        print('Var1: ', var1)
        print('Var2: ', var2)

# testing 

In [28]:
df_test=pd.read_csv('~/data/testdata.csv')
df_test

,1_AIT_001_PV,1_AIT_002_PV,1_AIT_003_PV,1_AIT_004_PV,1_AIT_005_PV,1_FIT_001_PV,1_LT_001_PV,1_MV_001_STATUS,1_MV_002_STATUS,1_MV_003_STATUS,...,3_AIT_001_PV,3_AIT_002_PV,3_AIT_003_PV,3_AIT_004_PV,3_AIT_005_PV,3_FIT_001_PV,3_LT_001_PV,LEAK_DIFF_PRESSURE,TOTAL_CONS_REQUIRED_FLOW,"Attack LABLE (1:No Attack, -1:Attack)"
0,161.129,0.721464,11.7842,463.716,0.267211,1.890010,48.9329,2.0,1.0,1.0,...,0.0,8279.1,5.95083,743.356,0.594601,0.001333,64.5137,74.2107,1.92,1.0
1,161.129,0.721464,11.7842,463.716,0.267211,1.890010,48.9329,2.0,1.0,1.0,...,0.0,8279.1,5.95083,743.356,0.594601,0.001333,64.5137,74.2107,1.92,1.0
2,161.129,0.721464,11.7842,463.716,0.267211,1.890010,48.9329,2.0,1.0,1.0,...,0.0,8279.1,5.95083,743.356,0.594601,0.001333,64.5137,74.2107,1.92,1.0
3,161.129,0.721464,11.7842,463.716,0.267211,1.890010,48.9329,2.0,1.0,1.0,...,0.0,8279.1,5.95083,743.356,0.594601,0.001333,64.5137,74.2107,1.92,1.0
4,161.129,0.721464,11.7842,463.716,0.267211,1.890010,48.9329,2.0,1.0,1.0,...,0.0,8279.1,5.95083,743.356,0.594601,0.001333,64.5137,74.2107,1.92,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248395,176.467,0.613474,11.9281,430.680,0.228327,0.001214,66.1851,1.0,1.0,1.0,...,0.0,8279.1,13.83640,370.386,0.729505,0.595032,64.3879,66.1173,1.41,1.0
248396,176.467,0.613474,11.9281,430.680,0.228327,0.001214,66.1851,1.0,1.0,1.0,...,0.0,8279.1,13.83640,370.386,0.729505,0.595032,64.3879,66.1173,1.41,1.0
248397,176.467,0.613474,11.9281,430.680,0.228327,0.001214,66.1851,1.0,1.0,1.0,...,0.0,8279.1,13.83640,370.386,0.729505,0.595032,64.3879,66.1173,1.41,1.0
248398,176.467,0.613474,11.9281,430.680,0.228327,0.001214,66.1851,1.0,1.0,1.0,...,0.0,8279.1,13.83640,370.386,0.729505,0.595032,64.3879,66.1173,1.41,1.0


In [29]:
y_actual=df_test.iloc[:,-1]
sensors = [3,9,25,43]
df_test = df_test.iloc[:, sensors]
df_test

,1_AIT_004_PV,1_MV_003_STATUS,2_FIC_401_CO,2_LS_101_AH
0,463.716,1.0,20.2507,0.0
1,463.716,1.0,20.2507,0.0
2,463.716,1.0,20.2507,0.0
3,463.716,1.0,20.2507,0.0
4,463.716,1.0,20.2507,0.0
...,...,...,...,...
248395,430.680,1.0,19.2170,0.0
248396,430.680,1.0,19.2170,0.0
248397,430.680,1.0,19.2170,0.0
248398,430.680,1.0,19.2170,0.0


In [30]:
y_actual.value_counts()

 1.0    246888
-1.0      1512
Name: Attack LABLE (1:No Attack, -1:Attack), dtype: int64

In [34]:
X_test=df_test.iloc[:,2].values
Xtest=hankel.fit(X_test,30,0.5)  #size -> 1800x274
Xtest=Xtest.T  #size -> 274x1800
for i in range(optimal_k):
    Xtest_cluster=np.matmul(Xtest,clusters_V[i][:,:clusters_R[i]]) # size -> 274 x R
    var1=np.square(Xtest_cluster-centers[i])
    var2=np.matmul(weights[i],var1.T)
    widths=calc_width_height(weights[i])
    print("max radii",math.sqrt(np.amax(var2)))

max radii 19.532212729024153
max radii 12.55192395401656
max radii 15.650782177318247
max radii 7.160820069277709


In [35]:
print(widths)

[1.64776709e+04 1.19516471e+04 7.09636099e+03 8.43014983e+03
 4.23497394e+03 5.53233541e+03 3.69393626e+03 3.31270697e+03
 3.39265602e+03 2.23565368e+03 2.89733689e+03 1.96020018e+03
 2.21356769e+03 1.41876317e+03 1.66993271e+03 1.46482945e+03
 1.23889344e+03 1.19508220e+03 1.36320736e+03 1.06922520e+03
 1.05854891e+03 1.03656847e+03 1.21306299e+03 9.24552911e+02
 1.01374897e+03 8.26455627e+02 1.50555952e+03 7.28939721e+02
 5.49574173e+02 6.00962985e+02 6.32531354e+02 6.91668632e+02
 5.93258182e+02 5.42809724e+02 4.94031127e+02 4.00898026e+02
 3.80528706e+02 7.37863198e+02 4.57643982e+02 4.23548541e+02
 3.76220106e+02 4.30323061e+02 9.16909679e+02 5.42782739e+02
 2.70374643e+02 2.70196413e+02 3.08048346e+02 2.62284762e+02
 2.78997367e+02 2.61471400e+02 4.07686202e+02 2.20761677e+02
 3.23912672e+02 2.97418638e+02 3.98640844e+02 3.05316182e+02
 2.07008928e+02 2.84116585e+02 2.26631034e+02 1.93974466e+02
 1.95713040e+02 2.14700280e+02 1.96960922e+02 1.54692576e+02
 1.42893778e+02 1.947600